<a href="https://colab.research.google.com/github/Haimonti/FraudDetection/blob/Test/AFD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
import pandas as pd
import numpy as np
import matplotlib as plt
import torch.nn as nn
import torch.optim as optim
from sklearn.neural_network import MLPClassifier

In [40]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AFD/Bao_28_items_1990_2023_v3.csv')
print(df.keys())

Index(['gvkey', 'datadate', 'fyear', 'indfmt', 'consol', 'popsrc', 'datafmt',
       'tic', 'cusip', 'conm', 'curcd', 'fyr', 'act', 'ap', 'at', 'ceq', 'che',
       'cogs', 'csho', 'dlc', 'dltis', 'dltt', 'dp', 'ib', 'invt', 'ivao',
       'ivst', 'lct', 'lt', 'ni', 'ppegt', 'ppent', 'pstk', 're', 'rect',
       'sale', 'sstk', 'txp', 'txt', 'xint', 'cik', 'costat', 'prcc_f',
       'conml', 'sic'],
      dtype='object')


In [41]:
# Display the resulting DataFrame
dh=df.head()
dh

,gvkey,datadate,fyear,indfmt,consol,popsrc,datafmt,tic,cusip,conm,...,sale,sstk,txp,txt,xint,cik,costat,prcc_f,conml,sic
0,1003,1990-01-31,1989,INDL,C,D,STD,ANTQ,000354100,A.A. IMPORTING CO INC,...,19.267,0.0,0.000,-1.37,0.702,730052.0,I,0.125000,A.A. Importing Co Inc,5712
1,1004,1990-05-31,1989,INDL,C,D,STD,AIR,000361105,AAR CORP,...,444.875,0.0,3.040,12.50,9.989,1750.0,A,21.249998,AAR Corp,5080
2,1004,1991-05-31,1990,INDL,C,D,STD,AIR,000361105,AAR CORP,...,466.542,0.0,4.019,6.55,10.073,1750.0,A,14.124999,AAR Corp,5080
3,1004,1992-05-31,1991,INDL,C,D,STD,AIR,000361105,AAR CORP,...,422.657,0.0,4.981,3.60,8.356,1750.0,A,12.874998,AAR Corp,5080
4,1004,1993-05-31,1992,INDL,C,D,STD,AIR,000361105,AAR CORP,...,382.780,0.0,2.394,-2.20,8.107,1750.0,A,13.499999,AAR Corp,5080


In [42]:
#Sort by gvkey, ticker and datadate
df.sort_values(by=['gvkey','datadate'], inplace=True)
df.drop_duplicates(subset=['gvkey', 'datadate'], keep='first', inplace=True)

In [43]:
#Count observations in data set
rows_count=len(df)
print(rows_count)

327848


In [44]:
#Identify Banks
df['Bank'] = (df['sic'] >= 6000) & (df['sic'] <= 6999)
df['Bank'] = df['Bank'].astype(int)

In [28]:
# Remove rows where Total Assets (at) is missing
# df = df.dropna(subset=['at'])

In [45]:
#Count observations in data set without missing at
rows_count=len(df)
print(rows_count)

327848


In [46]:
#Count observations in data set with at = 0
rows_count_0 = (df['at'] == 0).sum()
print(rows_count_0)

1099


In [47]:
# Remove rows where at = 0
# df = df[df['at'] != 0]

In [48]:
#Count observations in data set with out 'at'=0
rows_count=len(df)
print(rows_count)

327848


In [49]:
#Sort by gvkey, ticker and datadate
df.sort_values(by=['gvkey','datadate'], inplace=True)

In [56]:
df_labels = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AFD/Annual_Labels2.csv')
print(df_labels.keys())

Index(['ID', 'CONAME', 'CIK', 'GVKEY', 'PERMNO', 'TICKER', 'CNUM', 'YEARA',
       'P_AAER', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11'],
      dtype='object')


In [57]:
# Merge dataframes on specified columns
merged_df = pd.merge(df, df_labels, left_on=['cik', 'fyear'], right_on=['CIK', 'YEARA'], how='left')

# Add a column with 1 for each record that is in df_labels
merged_df['Misstatements'] = merged_df['CIK'].notnull().astype(int)


# Display the merged dataframe
dh1=merged_df.head()
dh1

,gvkey,datadate,fyear,indfmt,consol,popsrc,datafmt,tic,cusip,conm,...,GVKEY,PERMNO,TICKER,CNUM,YEARA,P_AAER,Unnamed: 9,Unnamed: 10,Unnamed: 11,Misstatements
0,1003,1990-01-31,1989,INDL,C,D,STD,ANTQ,000354100,A.A. IMPORTING CO INC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,1004,1990-05-31,1989,INDL,C,D,STD,AIR,000361105,AAR CORP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,1004,1991-05-31,1990,INDL,C,D,STD,AIR,000361105,AAR CORP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,1004,1992-05-31,1991,INDL,C,D,STD,AIR,000361105,AAR CORP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,1004,1993-05-31,1992,INDL,C,D,STD,AIR,000361105,AAR CORP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [59]:
column_name = 'Misstatements'
count_ones = merged_df[column_name].eq(1).sum()
print(count_ones)

1403


In [60]:
# Save the merged dataframe to a CSV file
merged_df.to_csv('merged_data2.csv', index=False)

In [ ]:
#1 changes in working capital accruals
     #wc=(act-che)-(lct-dlc-txp)
     #ch_wc = wc - lag_wc)
     #dch_wc = ch_wc*2/(at-lag_at))

# Calculate lags
df['lag_gvkey'] = df['gvkey'].shift(1)
df['lag_fyear'] = df['fyear'].shift(1)
df['lag_cik'] = df['cik'].shift(1)
df['lag_at'] = df['at'].shift(1)

# Set lag variables to missing if gvkey changes
df.loc[df['lag_gvkey'] != df['gvkey'], ['lag_fyear', 'lag_cik', 'lag_at']] = pd.NA

# Calculate changes in working capital accruals
df['txp'] = df['txp'].fillna(0) # Fill txp with 0 if missing
df['wc'] = (df['act'] - df['che']) - (df['lct'] - df['dlc'] - df['txp'])
df['lag_wc'] = df['wc'].shift(1)

# Calculate change in working capital
df['ch_wc'] = df.apply(lambda row: row['wc'] - row['lag_wc'] if row['lag_gvkey'] == row['gvkey'] and row['lag_fyear'] == row['fyear'] - 1 else pd.NA, axis=1)

# Calculate dch_wc
df['dch_wc'] = df['ch_wc'] * 2 / (df['at'] + df['lag_at'])

# Drop unnecessary columns if needed
df = df.drop(['lag_gvkey', 'lag_fyear', 'lag_cik', 'lag_at', 'lag_wc'], axis=1)

# Display the resulting DataFrame
dh=df.head()
dh

,gvkey,datadate,fyear,indfmt,consol,popsrc,datafmt,tic,cusip,conm,...,xint,cik,costat,prcc_f,conml,sic,Bank,wc,ch_wc,dch_wc
0,1003,1990-01-31,1989,INDL,C,D,STD,ANTQ,000354100,A.A. IMPORTING CO INC,...,0.702,730052.0,I,0.125000,A.A. Importing Co Inc,5712,0,2.499,<NA>,<NA>
1,1004,1990-05-31,1989,INDL,C,D,STD,AIR,000361105,AAR CORP,...,9.989,1750.0,A,21.249998,AAR Corp,5080,0,217.749,<NA>,<NA>
2,1004,1991-05-31,1990,INDL,C,D,STD,AIR,000361105,AAR CORP,...,10.073,1750.0,A,14.124999,AAR Corp,5080,0,208.138,-9.611,-0.025013
3,1004,1992-05-31,1991,INDL,C,D,STD,AIR,000361105,AAR CORP,...,8.356,1750.0,A,12.874998,AAR Corp,5080,0,223.035,14.897,0.038429
4,1004,1993-05-31,1992,INDL,C,D,STD,AIR,000361105,AAR CORP,...,8.107,1750.0,A,13.499999,AAR Corp,5080,0,218.563,-4.472,-0.011761


In [ ]:
#2 changes in RSST_accruals
     #nco=(at-act-ivao)-(lt-lct-dltt)
     #ch_nco=nco-lag_nco
     #fin=(ivst+ivao)-(dltt+dlc+pstk)
     #ch_fin=fin-lag_fin
     #ch_rsst=(ch_wc+ch_nco+ch_fin)*2/(at+lag_at)

# Calculate lags
df['lag_gvkey'] = df['gvkey'].shift(1)
df['lag_fyear'] = df['fyear'].shift(1)
df['lag_cik'] = df['cik'].shift(1)
df['lag_at'] = df['at'].shift(1)

# Step 1
df['ivao'].fillna(0, inplace=True)
df['nco'] = (df['at'] - df['act'] - df['ivao']) - (df['lt'] - df['lct'] - df['dltt'])
df['lag_nco'] = df['nco'].shift()

# Step 2
df['ch_nco'] = df.apply(lambda row: row['nco'] - row['lag_nco'] if row['lag_gvkey'] == row['gvkey'] and row['lag_fyear'] == row['fyear'] - 1 else None, axis=1)

# Step 3
df['ivst'].fillna(0, inplace=True)
df['pstk'].fillna(0, inplace=True)
df['fin'] = (df['ivst'] + df['ivao']) - (df['dltt'] + df['dlc'] + df['pstk'])
df['lag_fin'] = df['fin'].shift()

# Step 4
df['ch_fin'] = df.apply(lambda row: row['fin'] - row['lag_fin'] if row['lag_gvkey'] == row['gvkey'] and row['lag_fyear'] == row['fyear'] - 1 else None, axis=1)

# Step 5
df['ch_rsst'] = (df['ch_wc'] + df['ch_nco'] + df['ch_fin']) * 2 / (df['at'] + df['lag_at'])

# Display the resulting DataFrame
dh=df.head()
dh




,gvkey,datadate,fyear,indfmt,consol,popsrc,datafmt,tic,cusip,conm,...,lag_fyear,lag_cik,lag_at,nco,lag_nco,ch_nco,fin,lag_fin,ch_fin,ch_rsst
0,1003,1990-01-31,1989,INDL,C,D,STD,ANTQ,000354100,A.A. IMPORTING CO INC,...,NaN,NaN,NaN,0.919,NaN,NaN,-4.525,NaN,NaN,<NA>
1,1004,1990-05-31,1989,INDL,C,D,STD,AIR,000361105,AAR CORP,...,1989.0,730052.0,10.109,43.118,0.919,NaN,-72.323,-4.525,NaN,<NA>
2,1004,1991-05-31,1990,INDL,C,D,STD,AIR,000361105,AAR CORP,...,1989.0,1750.0,388.521,40.570,43.118,-2.548,-52.464,-72.323,19.859,0.02004
3,1004,1992-05-31,1991,INDL,C,D,STD,AIR,000361105,AAR CORP,...,1990.0,1750.0,379.958,34.015,40.570,-6.555,-57.582,-52.464,-5.118,0.008317
4,1004,1993-05-31,1992,INDL,C,D,STD,AIR,000361105,AAR CORP,...,1991.0,1750.0,395.351,29.905,34.015,-4.110,-59.113,-57.582,-1.531,-0.026596


In [ ]:
#3 changes in receivables
    #ch_rec=rect-lag_rect
    #dch_rec=ch_rec*2/(at+lag_at)


df['lag_rect'] = df['rect'].shift(1)
df['lag_at'] = df['at'].shift(1)

# Calculate changes in receivables
df['ch_rec'] = df.apply(lambda row: row['rect'] - row['lag_rect'] if row['lag_gvkey'] == row['gvkey'] and row['lag_fyear'] == row['fyear'] - 1 else None, axis=1)

# Calculate dch_rec
df['dch_rec'] = df['ch_rec'] * 2 / (df['at'] + df['lag_at'])

# Display the resulting DataFrame
dh=df.head()
dh



,gvkey,datadate,fyear,indfmt,consol,popsrc,datafmt,tic,cusip,conm,...,nco,lag_nco,ch_nco,fin,lag_fin,ch_fin,ch_rsst,lag_rect,ch_rec,dch_rec
0,1003,1990-01-31,1989,INDL,C,D,STD,ANTQ,000354100,A.A. IMPORTING CO INC,...,0.919,NaN,NaN,-4.525,NaN,NaN,<NA>,NaN,NaN,NaN
1,1004,1990-05-31,1989,INDL,C,D,STD,AIR,000361105,AAR CORP,...,43.118,0.919,NaN,-72.323,-4.525,NaN,<NA>,1.373,NaN,NaN
2,1004,1991-05-31,1990,INDL,C,D,STD,AIR,000361105,AAR CORP,...,40.570,43.118,-2.548,-52.464,-72.323,19.859,0.02004,93.581,-2.343,-0.006098
3,1004,1992-05-31,1991,INDL,C,D,STD,AIR,000361105,AAR CORP,...,34.015,40.570,-6.555,-57.582,-52.464,-5.118,0.008317,91.238,-1.457,-0.003759
4,1004,1993-05-31,1992,INDL,C,D,STD,AIR,000361105,AAR CORP,...,29.905,34.015,-4.110,-59.113,-57.582,-1.531,-0.026596,89.781,-20.932,-0.055048


In [ ]:
#4 changes in inventories
    #ch_inv=invt-lag_invt
    #dch_inv=ch_inv*2/(at+lag_at)
# Create a lagged variable

df['lag_invt'] = df['invt'].shift(1)

# Calculate ch_inv based on conditions
df['ch_inv'] = df.apply(lambda row: row['invt'] - row['lag_invt'] if row['lag_gvkey'] == row['gvkey'] and row['lag_fyear'] == row['fyear'] - 1 else None, axis=1)

# Calculate dch_inv
df['dch_inv'] = df['ch_inv'] * 2 / (df['at'] + df['lag_at'])

# Drop intermediate columns if needed
df = df.drop(['lag_invt', 'ch_inv'], axis=1)

# Display the resulting DataFrame
dh=df.head()
dh

,gvkey,datadate,fyear,indfmt,consol,popsrc,datafmt,tic,cusip,conm,...,lag_nco,ch_nco,fin,lag_fin,ch_fin,ch_rsst,lag_rect,ch_rec,dch_rec,dch_inv
0,1003,1990-01-31,1989,INDL,C,D,STD,ANTQ,000354100,A.A. IMPORTING CO INC,...,NaN,NaN,-4.525,NaN,NaN,<NA>,NaN,NaN,NaN,NaN
1,1004,1990-05-31,1989,INDL,C,D,STD,AIR,000361105,AAR CORP,...,0.919,NaN,-72.323,-4.525,NaN,<NA>,1.373,NaN,NaN,NaN
2,1004,1991-05-31,1990,INDL,C,D,STD,AIR,000361105,AAR CORP,...,43.118,-2.548,-52.464,-72.323,19.859,0.02004,93.581,-2.343,-0.006098,-0.015657
3,1004,1992-05-31,1991,INDL,C,D,STD,AIR,000361105,AAR CORP,...,40.570,-6.555,-57.582,-52.464,-5.118,0.008317,91.238,-1.457,-0.003759,0.044488
4,1004,1993-05-31,1992,INDL,C,D,STD,AIR,000361105,AAR CORP,...,34.015,-4.110,-59.113,-57.582,-1.531,-0.026596,89.781,-20.932,-0.055048,-0.002217


In [ ]:
#5 percentage of soft assets
    #soft_assets=(at-ppent-che)/at

# Calculate soft_assets
df['soft_assets'] = (df['at'] - df['ppent'] - df['che']) / df['at']


# Display the resulting DataFrame
dh=df.head()
dh

,gvkey,datadate,fyear,indfmt,consol,popsrc,datafmt,tic,cusip,conm,...,ch_nco,fin,lag_fin,ch_fin,ch_rsst,lag_rect,ch_rec,dch_rec,dch_inv,soft_assets
0,1003,1990-01-31,1989,INDL,C,D,STD,ANTQ,000354100,A.A. IMPORTING CO INC,...,NaN,-4.525,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,0.866950
1,1004,1990-05-31,1989,INDL,C,D,STD,AIR,000361105,AAR CORP,...,NaN,-72.323,-4.525,NaN,<NA>,1.373,NaN,NaN,NaN,0.826303
2,1004,1991-05-31,1990,INDL,C,D,STD,AIR,000361105,AAR CORP,...,-2.548,-52.464,-72.323,19.859,0.02004,93.581,-2.343,-0.006098,-0.015657,0.829013
3,1004,1992-05-31,1991,INDL,C,D,STD,AIR,000361105,AAR CORP,...,-6.555,-57.582,-52.464,-5.118,0.008317,91.238,-1.457,-0.003759,0.044488,0.836553
4,1004,1993-05-31,1992,INDL,C,D,STD,AIR,000361105,AAR CORP,...,-4.110,-59.113,-57.582,-1.531,-0.026596,89.781,-20.932,-0.055048,-0.002217,0.840321


In [ ]:
#6 percentage change in cash sales
    #cs=sale-(rect-lag_rect)
    #ch_cs=(cs-lag_cs)/lag_cs






In [ ]:
#7 change in cash margin
    #cmm=(cogs-(invt-lag_invt)+(aplag_ap))/(sale-(rect-lag_rect))
    #ch_cm=(cm-lag_cm)/lag_cm





In [ ]:
#8 change in return on assets
    #ch_roa=roa - lag_roa


# First Block
df['roa'] = df['ni'] * 2 / (df['at'] + df['lag_at'].shift(1))
df['lag_roa'] = df['roa'].shift(1)

# Second Block
df['ch_roa'] = None
mask = (df['lag_gvkey'] == df['gvkey']) & (df['lag_fyear'] == (df['fyear'] - 1))
df.loc[mask, 'ch_roa'] = df['roa'] - df['lag_roa']

# Third Block
df.loc[~mask, 'ch_roa'] = None


In [ ]:
#9 actual issuance
    #sstk>0 --> issue=1
    #dltis>0 --> issue=1


# Create a new column 'issue' based on conditions
df['issue'] = None  # Initialize issue column with None

mask1 = df['sstk'] > 0
mask2 = df['dltis'] > 0
mask3 = df['sstk'].isnull() & df['dltis'].isnull()

df.loc[mask1, 'issue'] = 1
df.loc[mask2, 'issue'] = 1
df.loc[mask3, 'issue'] = None
df['issue'].fillna(0, inplace=True)




In [ ]:
#10 Book-to-market
    #bm=ceq/(prcc_f*csho)


df['bm'] = df['ceq'] / (df['prcc_f'] * df['csho'])


In [ ]:
#11 Depreciation Index (Ratio from Beneish 1999)
    #dpi=(lag_dp/(lag_dp+lag_ppent))/(dp/(dp+ppent))

# Create lagged variables dp and ppent
df['lag_dp'] = df['dp'].shift()
df['lag_ppent'] = df['ppent'].shift()

# Calculate dpi based on conditions
df['dpi'] = None  # Initialize dpi column with None

mask1 = (df['lag_gvkey'] == df['gvkey']) & (df['lag_fyear'] == df['fyear'] - 1)
mask2 = (df['lag_gvkey'] != df['gvkey']) | (df['lag_fyear'] != df['fyear'] - 1)

df.loc[mask1, 'dpi'] = (df['lag_dp'] / (df['lag_dp'] + df['lag_ppent'])) / \
                       (df['dp'] / (df['dp'] + df['ppent']))
df.loc[mask2, 'dpi'] = None

# Drop the lagged variables if you don't need them anymore
df = df.drop(columns=['lag_dp', 'lag_ppent'])

# Display the resulting DataFrame
dh=df.head()
dh

,gvkey,datadate,fyear,indfmt,consol,popsrc,datafmt,tic,cusip,conm,...,lag_cm,ch_cm,ch_cs,roa,lag_roa,ch_roa,ch_ib,issue,bm,dpi
0,1003,1990-01-31,1989,INDL,C,D,STD,ANTQ,000354100,A.A. IMPORTING CO INC,...,None,None,None,NaN,NaN,None,None,1,-1.240403,None
1,1004,1990-05-31,1989,INDL,C,D,STD,AIR,000361105,AAR CORP,...,NaN,None,None,NaN,NaN,None,None,1,0.554652,None
2,1004,1991-05-31,1990,INDL,C,D,STD,AIR,000361105,AAR CORP,...,NaN,None,None,0.075890,NaN,NaN,-0.028248,0,0.863306,0.882711
3,1004,1992-05-31,1991,INDL,C,D,STD,AIR,000361105,AAR CORP,...,0.208092,None,None,0.025565,0.075890,-0.050324,-0.012333,1,0.961101,0.968053
4,1004,1993-05-31,1992,INDL,C,D,STD,AIR,000361105,AAR CORP,...,0.240518,None,None,0.000760,0.025565,-0.024806,-0.025607,0,0.881454,1.084524


In [ ]:
#12 Retained earnings over assets
    #roa=ni*2/(at+lag_at)
    #reoa=re/at



In [ ]:
#13 Earnings before interest and tax (Ratios from Summers and Sweeney, 1998)
    #EBIT=(ni+xint+txt)/at

df['EBIT'] = (df['ni'] + df['xint'] + df['txt']) / df['at']


In [ ]:
#14 changes in free cash flow
    #ch_ib=(iblag_ib)*2/(at+lag_at)
    #ch_fcf=ch_ib-ch_rsst


df['lag_ib'] = df['ib'].shift()

# Create ch_ib based on conditions
df['ch_ib'] = None  # Initialize ch_ib column with None
mask1 = (df['lag_gvkey'] == df['gvkey']) & (df['lag_fyear'] == df['fyear'] - 1)
mask2 = (df['lag_gvkey'] != df['gvkey']) | (df['lag_fyear'] != df['fyear'] - 1)

df.loc[mask1, 'ch_ib'] = (df['ib'] - df['lag_ib']) * 2 / (df['at'] + df['lag_at'])
df.loc[mask2, 'ch_ib'] = None

# Drop the lag_ib column if you don't need it anymore
df = df.drop(columns=['lag_ib'])

# Display the resulting DataFrame
dh=df.head()
dh

,gvkey,datadate,fyear,indfmt,consol,popsrc,datafmt,tic,cusip,conm,...,lag_invt,cmm,cm,lag_cm,ch_cm,ch_cs,roa,lag_roa,ch_roa,ch_ib
0,1003,1990-01-31,1989,INDL,C,D,STD,ANTQ,000354100,A.A. IMPORTING CO INC,...,NaN,None,NaN,None,None,None,NaN,NaN,None,None
1,1004,1990-05-31,1989,INDL,C,D,STD,AIR,000361105,AAR CORP,...,6.935,None,NaN,NaN,None,None,NaN,NaN,None,None
2,1004,1991-05-31,1990,INDL,C,D,STD,AIR,000361105,AAR CORP,...,162.149,0.791908,0.208092,NaN,None,None,0.075890,NaN,NaN,-0.028248
3,1004,1992-05-31,1991,INDL,C,D,STD,AIR,000361105,AAR CORP,...,156.133,0.759482,0.240518,0.208092,None,None,0.025565,0.075890,-0.050324,-0.012333
4,1004,1993-05-31,1992,INDL,C,D,STD,AIR,000361105,AAR CORP,...,173.379,0.736634,0.263366,0.240518,None,None,0.000760,0.025565,-0.024806,-0.025607
